In [1]:
import sys
sys.path.insert(0, "../..")
import config as cfg
import gc

In [2]:
import pandas as pd
import numpy as np
from metrics import *
from helper import *

In [3]:
EXPERIMENT_FAMILY_NAME = 'blending'
EXPERIMENT_NAME = 'baseline'

In [4]:
model_names = {
    # 'logreg': ['baseline'],
    # 'mlp': ['baseline'],
    # 'svm': ['baseline'],
    # 'gbt': ['baseline'],
    # 'rf': ['baseline'],
    'catboost': ['baseline', 'only_real_cols', 'only_cat_cols']
}

In [5]:
train = pd.read_pickle(cfg.PREPARED_TRAIN_DATA_PATH)
oof_pred_probas = []
test_pred_probas = []

for model_name in model_names:
    for experiment_name in model_names[model_name]:
        oof_pred_probas.append(pd.read_pickle(os.path.join(cfg.OOF_PRED_PATH, model_name, f'{experiment_name}.pkl')))
        test_pred_probas.append(pd.read_pickle(os.path.join(cfg.TEST_PRED_PATH, model_name, f'{experiment_name}.pkl')))

In [6]:
def process_pred_proba(pred_probas: list) -> pd.DataFrame:
    first_df = pred_probas[0]
    blend_oof = pd.DataFrame(data=np.zeros(shape=first_df.shape), index=first_df.index, columns=first_df.columns)
    for pred_proba in pred_probas:
        pred_proba = pred_proba.rank() / (len(pred_proba) - 1)
        blend_oof += pred_proba
    blend_oof /= len(pred_probas)
    return blend_oof

In [7]:
blend_oof = process_pred_proba(oof_pred_probas)
blend_test = process_pred_proba(test_pred_probas)

In [8]:
tresholds = get_tresholds(train[cfg.TARGETS], blend_oof)
sample_submission = pd.read_csv(cfg.SAMPLE_SUBMISSION_PATH).set_index('ID')
submission = make_prediction(blend_test, tresholds, sample_submission)

0.6740305672632048


In [9]:
# 0.6902553424448903

# model_names = {
#     # 'logreg': ['baseline'],
#     # 'mlp': ['baseline'],
#     # 'svm': ['baseline'],
#     # 'gbt': ['baseline'],
#     # 'rf': ['baseline'],
#     'catboost': ['baseline', 'only_real_cols']
# }

In [11]:
submission_path = os.path.join(cfg.SUBMISSION_PATH, EXPERIMENT_FAMILY_NAME)
check_path(submission_path)
submission.to_csv(os.path.join(submission_path, f'{EXPERIMENT_NAME}.csv'))

blend_oof_path = os.path.join(cfg.OOF_PRED_PATH, EXPERIMENT_FAMILY_NAME)
check_path(blend_oof_path)
blend_oof.to_pickle(os.path.join(blend_oof_path, f'{EXPERIMENT_NAME}.pkl'))

blend_test_path = os.path.join(cfg.TEST_PRED_PATH, EXPERIMENT_FAMILY_NAME)
check_path(blend_test_path)
blend_test.to_pickle(os.path.join(blend_test_path, f'{EXPERIMENT_NAME}.pkl'))